In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/Users/sheethalsupriya/Documents/python_env/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/Users/sheethalsupriya/Documents/python_env/bin/python3'


In [2]:
pip install pyspark



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Flight Delay Analysis") \
    .master("local[*]") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 17:36:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("hdfs://localhost:9000/user/ssn/airline_data/*.csv")
df.show(5)
df.printSchema()


25/05/09 17:36:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+----

25/05/09 17:36:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, IATA_CODE_Reporting_Airline, Tail_Number, Flight_Number_Reporting_Airline, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsedTime, AirTime, Flights, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, FirstDepTime, TotalAddGTime, LongestAddGTime, DivAirportLandi

### Cause-Wise Delay Contribution

In [13]:
from pyspark.sql.functions import sum

print("=== Average Delay by Cause ===")
df.selectExpr(
    "avg(CarrierDelay) as Carrier", 
    "avg(WeatherDelay) as Weather",
    "avg(NASDelay) as NAS", 
    "avg(SecurityDelay) as Security", 
    "avg(LateAircraftDelay) as LateAircraft"
).show()



=== Average Delay by Cause ===


+------------------+-----------------+------------------+-------------------+------------------+
|           Carrier|          Weather|               NAS|           Security|      LateAircraft|
+------------------+-----------------+------------------+-------------------+------------------+
|24.706312018781155|4.273121953372636|13.532724950412563|0.12409135675092622|28.948086986987263|
+------------------+-----------------+------------------+-------------------+------------------+



### Total Number of Flights


In [15]:
print("=== Total Number of Flights ===")
print(df.count())



=== Total Number of Flights ===


7079061


### Flights per Day

In [16]:
print("=== Number of Flights per Day ===")
df.groupBy("FlightDate").count().orderBy("FlightDate").show()


=== Number of Flights per Day ===


+----------+-----+
|FlightDate|count|
+----------+-----+
|2024-01-01|17265|
|2024-01-02|18977|
|2024-01-03|18520|
|2024-01-04|18066|
|2024-01-05|18109|
|2024-01-06|16950|
|2024-01-07|18812|
|2024-01-08|18472|
|2024-01-09|16775|
|2024-01-10|16795|
|2024-01-11|18500|
|2024-01-12|18590|
|2024-01-13|15483|
|2024-01-14|17583|
|2024-01-15|18620|
|2024-01-16|16530|
|2024-01-17|16877|
|2024-01-18|18663|
|2024-01-19|18698|
|2024-01-20|15535|
+----------+-----+
only showing top 20 rows



### Most Active Airlines

In [17]:
print("=== Most Active Airlines ===")
df.groupBy("Reporting_Airline").count().orderBy("count", ascending=False).show()


=== Most Active Airlines ===


+-----------------+-------+
|Reporting_Airline|  count|
+-----------------+-------+
|               WN|1419419|
|               DL|1009174|
|               AA| 984306|
|               UA| 760451|
|               OO| 744658|
|               YX| 301465|
|               MQ| 279955|
|               NK| 261103|
|               AS| 245819|
|               B6| 240282|
|               OH| 227971|
|               F9| 208624|
|               9E| 200094|
|               G4| 117210|
|               HA|  78530|
+-----------------+-------+



### Top Destinations

In [18]:
print("=== Most Common Destination Airports ===")
df.groupBy("Dest").count().orderBy("count", ascending=False).show()


=== Most Common Destination Airports ===


+----+------+
|Dest| count|
+----+------+
| ATL|341825|
| DFW|313568|
| DEN|308609|
| ORD|280009|
| CLT|217545|
| LAX|194044|
| PHX|193542|
| LAS|189264|
| SEA|163707|
| LGA|162470|
| MCO|159578|
| BOS|143491|
| DCA|140015|
| SFO|135075|
| DTW|131360|
| EWR|128978|
| MSP|126793|
| JFK|123334|
| IAH|115961|
| SLC|113239|
+----+------+
only showing top 20 rows



### Average Air Time

In [19]:
print("=== Average Air Time of Flights ===")
df.select(avg("AirTime").alias("AvgAirTime")).show()


=== Average Air Time of Flights ===


+------------------+
|        AvgAirTime|
+------------------+
|114.99991586802305|
+------------------+



### Delays Overview

In [20]:
from pyspark.sql.functions import avg

df.select(
    avg("DepDelay").alias("AvgDepDelay"),
    avg("ArrDelay").alias("AvgArrDelay")
).show()


+------------------+-----------------+
|       AvgDepDelay|      AvgArrDelay|
+------------------+-----------------+
|12.677046863546439|7.098219058132468|
+------------------+-----------------+

